In [2]:
import pandas as pd

In [27]:
df = pd.read_csv('college.csv')

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup as bs

def preprocess(sentence):
    soup = bs(sentence, features="html.parser")
    sentence = soup.get_text()
    
    sentence = str(sentence).lower()
    sentence = sentence.replace('{html}',"")
    
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url = re.sub(r'http\S+', '', cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    
    tokenizer = RegexpTokenizer(r'\w+|[.,]')
    tokens = tokenizer.tokenize(rem_num)
    
    filtered_words = [w for w in tokens if not w in stopwords.words('russian')]

    processed_text = re.sub(r'\s+([.,])', r'\1', " ".join(filtered_words))
    
    processed_text = re.sub(r'\.\s*\.', '.', processed_text)
    
    return processed_text

In [19]:
df['description'] = df['description'].apply(preprocess)

In [20]:
words = ['Диплом колледжа', 'средне-специальное образование', 'Вакансия доступна выпускникам колледжей', 'диплом колледжа']

In [21]:
names = ['Автоэлектрик','Аналитик-консультант','Менеджер - оценщик', 'Эксперт-оценщик','Бухгалтер-стажер',
         'Менеджер по взысканию','Инженер по продажам','Менеджер-оценщик','Помощник оценщика', 'Кредитный менеджер', 'СММ Менеджер',
        'Консультант', 'Инспектор по взысканию', 'Специалист по взысканию', 'Системный администратор', 'Инспектор по взысканию','HR-менеджер','Специалист по учету кадров',
        'Менеджер по продажам', 'Офис-менеджер', 'Инженер по бурению', 'Директор по персоналу']

In [24]:
df_filtered = df_filtered[~df_filtered['description'].str.contains('|'.join(words), case=False, na=False)]

In [25]:
df_filtered

,name,description,response
0,Старший преподаватель,"внимание, конкурс филиал ргкп академия государ...","yes 1.ПРЕПОДАВАТЕЛЬ, ДОЦЕНТ, ПРОФЕССОР ИНФОРМА..."
1,Старший преподаватель,"внимание, конкурс филиал ргкп академия государ...","yes 1.ПРЕПОДАВАТЕЛЬ, ДОЦЕНТ, ПРОФЕССОР ИНФОРМА..."
2,Старший преподаватель по переподготовке и повы...,"назар аударыңыз, конкурс қазақстан республикас...","yes 1.ПРЕПОДАВАТЕЛЬ, ДОЦЕНТ, ПРОФЕССОР ИНФОРМА..."
3,Преподаватель,"назар аударыңыз, конкурс қазақстан республикас...","yes 1.ПРЕПОДАВАТЕЛЬ, ДОЦЕНТ, ПРОФЕССОР ИНФОРМА..."
4,Преподаватель английского языка,кафедру английского языка требуются сотрудники...,no \n
...,...,...,...
642,Доцент кафедры,кафедра архитектуры ед. кафедра геодезии карто...,no \n\n\nThis description outlines requirement...
643,Профессор кафедры «Геодезии и картографии»,"наличие ученой степени кандидата наук, доктора...",no \n
644,Халықаралық ынтымақтастық және академиялық ұтқ...,міндеттері шетелдік әріптестермен ынтымақтасты...,no \n
645,Менеджер,требования образование высшееспециальность док...,no \n


In [297]:
df = uni

In [28]:
API_URL = "http://51.20.96.227/api/v1/prediction/c01b4b2d-07e1-4c85-90f9-34f9417bce63"

In [29]:
def query(payload):
    response = requests.post(API_URL, json=payload)
    return response.json()

In [30]:
df= df.drop('response', axis=1)

In [31]:
if 'response' not in df.columns:
    df['response'] = ""

In [ ]:
import pandas as pd
import aiohttp
import asyncio

CSV_OUTPUT_PATH = 'college.csv'
BATCH_SAVE_SIZE = 50 

unprocessed_df = df[df['response'].isna() | (df['response'] == "")]
total_records = len(unprocessed_df)
processed_count = 0

async def query(session, payload):
    try:
        async with session.post(API_URL, json=payload) as response:
            response.raise_for_status()  
            return await response.json()
    except Exception as e:
        print(f"Request failed: {e}")
        return None

async def process_records(unprocessed_df):
    global processed_count

    async with aiohttp.ClientSession() as session:
        tasks = []
        indices = []

        for index, row in unprocessed_df.iterrows():
            tasks.append(query(session, {"question": row['description']}))
            indices.append(index)

            if len(tasks) >= BATCH_SAVE_SIZE:
                responses = await asyncio.gather(*tasks)
                for idx, response in enumerate(responses):
                    if response is not None:
                        df.at[indices[idx], 'response'] = response.get('text', '')

                tasks.clear()
                indices.clear()

                processed_count += BATCH_SAVE_SIZE
                remaining_records = total_records - processed_count
                print(f"Обработано {processed_count} из {total_records} записей. Осталось: {remaining_records}")
                df.to_csv(CSV_OUTPUT_PATH, index=False)

        if tasks:
            responses = await asyncio.gather(*tasks)
            for idx, response in enumerate(responses):
                if response is not None:
                    df.at[indices[idx], 'response'] = response.get('text', '')

            processed_count += len(tasks)
            remaining_records = total_records - processed_count
            print(f"Обработано {processed_count} из {total_records} записей. Осталось: {remaining_records}")

        df.to_csv(CSV_OUTPUT_PATH, index=False)

loop = asyncio.get_event_loop()
if loop.is_running():
    await process_records(unprocessed_df)
else:
    loop.run_until_complete(process_records(unprocessed_df))

print(f"Все результаты сохранены в '{CSV_OUTPUT_PATH}'.")

Обработано 50 из 647 записей. Осталось: 597
Обработано 100 из 647 записей. Осталось: 547
Обработано 150 из 647 записей. Осталось: 497
Обработано 200 из 647 записей. Осталось: 447
Обработано 250 из 647 записей. Осталось: 397
Обработано 300 из 647 записей. Осталось: 347
Обработано 350 из 647 записей. Осталось: 297
Обработано 400 из 647 записей. Осталось: 247
Обработано 450 из 647 записей. Осталось: 197
Обработано 500 из 647 записей. Осталось: 147
Обработано 550 из 647 записей. Осталось: 97


In [441]:
df_f = df[df['response']!='no \n']

In [442]:
df_f = df_f[df_f['response']!='no \n\n\n'] ##

In [443]:
df_f = df_f[df_f['response']!='no \n\n\nLet me know if you have another job description to classify! \n'] ##

In [47]:
selskoe = pd.read_excel('after_extract/uni/Преподаватель вуза - педагогика.xlsx')

In [48]:
selskoe['description'] = selskoe['description'].apply(lambda x: preprocess(x))

In [49]:
selskoe.to_excel('after_extract/uni/Преподаватель вуза - педагогика.xlsx')

In [51]:
f_113 = pd.read_excel('before_extract/113.xlsx')

In [53]:
f_113['description'] = f_113['description'].apply(preprocess)

In [55]:
f_113.to_excel('before_extract/113.xlsx')

In [77]:
df_113 = pd.read_excel('fix_for_extract/113.xlsx')

In [62]:
f_274 = pd.read_excel('fix_for_extract/274.xlsx')

In [64]:
f_274['description'] = f_274['description'].apply(preprocess)

In [65]:
f_274.to_excel('fix_for_extract/274.xlsx')

In [72]:
f_282 = pd.read_excel('fix_for_extract/282.xlsx')

In [69]:
f_282['description'] = df['description'].apply(preprocess)

In [70]:
f_282.to_excel('fix_for_extract/282.xlsx')

In [91]:
f_263 = pd.read_excel('fix_for_extract/263.xlsx')

In [84]:
f_263['description'] = f_263['description'].apply(lambda x: preprocess(x))

In [85]:
f_263.to_excel('fix_for_extract/263.xlsx')

In [86]:
df_402 = pd.read_excel('fix_for_extract/400-professions (2).xlsx')

In [89]:
len(df_402['ID'].value_counts())

400

In [103]:
f_263.rename(columns={'profession_id':'class'}, inplace=True)

In [97]:
f_327 = pd.read_excel('fix_for_extract/327.xlsx')

In [95]:
f_327['description'] = f_327['description'].apply(preprocess)

In [96]:
f_327.to_excel('fix_for_extract/327.xlsx')

In [99]:
f_327['profession_id'] = 327

In [101]:
f_327.rename(columns={'profession_id':'class'}, inplace=True)

In [102]:
f_327

,Unnamed: 0,id,name,description,class
0,0.0,22859.0,Инженер по стандартизации и сертификации проду...,обязанности планирование организация работ ста...,327
1,1.0,22859.0,Инженер по стандартизации и сертификации проду...,обязанности осуществляет обязательный нормализ...,327
2,2.0,22859.0,Инженер по стандартизации и сертификации проду...,обязанности оформление документов получения од...,327
3,3.0,22859.0,Инженер по стандартизации и сертификации проду...,обязанности осуществлять поиск материалов отеч...,327
4,4.0,136869.0,Инженер по стандартизации и метрологии,обязанности метрологическое сопровождение обор...,327
...,...,...,...,...,...
195,NaN,NaN,Специалист по аудиту производственных процессов,Обязанности: Проводит аудиты производственных ...,327
196,NaN,NaN,Инженер по разработке нормативной документации,Обязанности: Разрабатывает и актуализирует нор...,327
197,NaN,NaN,Специалист по управлению качеством,"Обязанности: Контролирует качество продукции, ...",327
198,NaN,NaN,Техник по сертификации и проверке документации,Обязанности: Ведет документацию по сертификаци...,327


In [112]:
f_338 = pd.read_excel('fix_for_extract/338.xlsx')

In [106]:
f_338['description'] = f_338['description'].apply(preprocess)

In [108]:
f_338.to_excel('fix_for_extract/338.xlsx')

In [114]:
f_338['profession_id'] = 338

In [116]:
f_338.rename(columns={'profession_id':'class'}, inplace=True)

In [344]:
f_373

,Unnamed: 0,id,name,description,class
0,0.0,166163.0,Инструктор-координатор по обучению службы авиа...,"обязанности планирование, организация, проведе...",373
1,1.0,40129.0,Инспектор Службы авиационной безопасности,обязанности обеспечивать охрану вс ангаре стоя...,373
2,2.0,40129.0,Инспектор Службы авиационной безопасности,обязанности осуществляет круглосуточную охрану...,373
3,3.0,8607.0,Инструктор по авиационной безопасности,года международный аэропорт алматы вошёл соста...,373
4,4.0,8607.0,Инструктор по авиационной безопасности,года международный аэропорт алматы вошёл соста...,373
...,...,...,...,...,...
194,NaN,NaN,Инспектор по логистике запасных частей,Контроль за поставками и учетом запасных часте...,373
195,NaN,NaN,Инспектор по проверке соблюдения инструкций,Контроль выполнения инструкций по безопасности...,373
196,NaN,NaN,Инспектор по качеству связи,Оценка качества связи и коммуникаций на всех э...,373
197,NaN,NaN,Инспектор по организации парковки воздушных судов,Контроль за процессом парковки воздушных судов...,373
